# Precomputation of open set performances

Used in Notebook 25 on Manuscript Section 1.

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

from NegativeClassOptimization import utils, preprocessing, ml

/home/eugen/miniconda3/envs/nco/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/eugen/miniconda3/envs/nco/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/eugen/miniconda3/envs/nco/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [2]:
experiment_ids = ["11", "13", "14"]
df = utils.MLFlowTaskAPI.mlflow_results_as_dataframe(experiment_ids, run_name="dev-v0.1.2-3-with-replicates", classify_tasks=True)

KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.


In [3]:
print(df.columns)
print(df.shape)
df

Index(['sample_train', 'load_from_miniabsolut_split_seed', 'optimizer_type',
       'ag_neg', 'batch_size', 'epochs', 'num_hidden_units', 'N_closed',
       'N_train', 'split_id', 'ag_pos', 'swa', 'shuffle_antigen_labels',
       'input_dim', 'learning_rate', 'weight_decay', 'momentum', 'seed_id',
       'load_from_miniabsolut', 'acc_closed', 'train_loss', 'roc_auc_closed',
       'f1_closed', 'avg_precision_closed', 'precision_closed', 'test_loss',
       'recall_closed', 'mlflow.source.git.commit', 'mlflow.note.content',
       'mlflow.source.type', 'mlflow.runName', 'mlflow.source.name',
       'mlflow.log-model.history', 'mlflow.user', 'experiment', 'run_id',
       'task', 'split_seed'],
      dtype='object')
(1200, 38)


,sample_train,load_from_miniabsolut_split_seed,optimizer_type,ag_neg,batch_size,epochs,num_hidden_units,N_closed,N_train,split_id,...,mlflow.note.content,mlflow.source.type,mlflow.runName,mlflow.source.name,mlflow.log-model.history,mlflow.user,experiment,run_id,task,split_seed
0,None,4,Adam,1H0D,64,50,10,10000,30000,0,...,1OB1 vs 1H0D,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12a_train_SN10_clean.py,"[{""run_id"": ""45e7f26f9d494b7cb2302f6f13c4bf55""...",eugen,11,45e7f26f9d494b7cb2302f6f13c4bf55,1v1,4
1,None,4,Adam,3VRL,64,50,10,10000,30000,0,...,1OB1 vs 3VRL,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12a_train_SN10_clean.py,"[{""run_id"": ""32f8fd48823c40c5bba365c9243e02de""...",eugen,11,32f8fd48823c40c5bba365c9243e02de,1v1,4
2,None,4,Adam,1FBI,64,50,10,10000,30000,0,...,1OB1 vs 1FBI,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12a_train_SN10_clean.py,"[{""run_id"": ""4f0eb2a80be645e3b6852c649537a693""...",eugen,11,4f0eb2a80be645e3b6852c649537a693,1v1,4
3,None,4,Adam,1WEJ,64,50,10,10000,30000,0,...,1OB1 vs 1WEJ,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12a_train_SN10_clean.py,"[{""run_id"": ""6c6537db1ea246da8ab39ab5b5a8749b""...",eugen,11,6c6537db1ea246da8ab39ab5b5a8749b,1v1,4
4,None,4,Adam,1NSN,64,50,10,10000,30000,0,...,1OB1 vs 1NSN,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12a_train_SN10_clean.py,"[{""run_id"": ""87a555c5f8fb4a92b497369e3113f76b""...",eugen,11,87a555c5f8fb4a92b497369e3113f76b,1v1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,None,None,Adam,1FBI_95low,64,50,10,10000,30000,0,...,1FBI_high vs 1FBI_95low,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12d_train_SN10_clean_high_loose...,"[{""run_id"": ""f0ab3615846141619a6e7e8870541f65""...",eugen,14,f0ab3615846141619a6e7e8870541f65,high_vs_95low,42
196,None,None,Adam,1OB1_looser,64,50,10,10000,30000,0,...,1OB1_high vs 1OB1_looser,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12d_train_SN10_clean_high_loose...,"[{""run_id"": ""c7fa79fc49f94647a5bd47ba5de0fefa""...",eugen,14,c7fa79fc49f94647a5bd47ba5de0fefa,high_vs_looser,42
197,None,None,Adam,2YPV_95low,64,50,10,10000,30000,0,...,2YPV_high vs 2YPV_95low,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12d_train_SN10_clean_high_loose...,"[{""run_id"": ""ac8861eb8c9a446c9d20408d034be3ce""...",eugen,14,ac8861eb8c9a446c9d20408d034be3ce,high_vs_95low,42
198,None,None,Adam,3VRL_looser,64,50,10,10000,30000,0,...,3VRL_high vs 3VRL_looser,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12d_train_SN10_clean_high_loose...,"[{""run_id"": ""c42fd3c99dcd44df971f9209e3dfad1c""...",eugen,14,c42fd3c99dcd44df971f9209e3dfad1c,high_vs_looser,42


In [4]:
from enum import Enum
from typing import Optional


class ClassificationTaskType(Enum):
    ONE_VS_ONE = 1
    ONE_VS_NINE = 2
    HIGH_VS_95LOW = 3
    HIGH_VS_LOOSER = 4


class ClassificationTask:

    def __init__(
        self,
        task_type: ClassificationTaskType,
        ag_pos: str,
        seed_id: int,
        split_id: int,
        ag_neg: Optional[str] = "auto",
        ):
        
        if task_type == ClassificationTaskType.ONE_VS_ONE and ag_neg == "auto":
            raise ValueError("ag_neg must be specified for ONE_VS_ONE task type")
        elif task_type != ClassificationTaskType.ONE_VS_ONE and ag_neg != "auto":
            raise ValueError("ag_neg must be set to 'auto' / not set for non-ONE_VS_ONE task type")

        self.task_type = task_type
        self.ag_pos = ag_pos
        self.seed_id = seed_id
        self.split_id = split_id
        self.ag_neg = ag_neg


    def __str__(self):
        if self.ag_neg is None:
            return f"{self.task_type.name}__{self.ag_pos}__{self.seed_id}__{self.split_id}"
        else:
            return f"{self.task_type.name}__{self.ag_pos}__{self.ag_neg}__{self.seed_id}__{self.split_id}"
    

    def __repr__(self):
        return str(self)


    def init_from_str(task_str: str):
        """
        Returns a Task object from a string.
        """
        task_type_name, ag_pos, ag_neg, seed_id, split_id = task_str.split("__")
        return ClassificationTask(
            task_type=ClassificationTaskType[task_type_name],
            ag_pos=ag_pos,
            ag_neg=ag_neg,
            seed_id=int(seed_id),
            split_id=int(split_id),
        )


    def __hash__(self) -> int:
        return hash(str(self))


    def __eq__(self, __value: object) -> bool:
        return str(self) == str(__value)


## Example
# t = ClassificationTask(
#     task_type=ClassificationTaskType.ONE_VS_ONE,
#     ag_pos="A",
#     ag_neg="B",
#     seed_id=0,
#     split_id=0,
# )

# t_inv = ClassificationTask.init_from_str(str(t))
# str(t_inv) == str(t)

True

In [ ]:
class FrozenMiniAbsolutMLLoader:
    """
    Loads the frozen MiniAbsolut dataset.
    """
    pass